In [14]:
import datetime
import itertools
import os
import pickle
import scipy
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from scipy.io import savemat
import scipy.io as sio
import sys
import importlib
from tempfile import TemporaryFile
import seaborn as sns
import sklearn as sk
import math
import argparse
from sklearn import preprocessing
from pypower.api import case30, case118
from pypower.api import opf, makeYbus, makeB
from pypower import idx_bus, idx_gen, ppoption
from torch.nn import Linear, ReLU
from torch_geometric.nn import GCNConv, Sequential
from torch.utils.data import TensorDataset, DataLoader
def str_to_bool(value):
    if isinstance(value, bool):
        return value
    if value.lower() in {'false', 'f', '0', 'no', 'n'}:
        return False
    elif value.lower() in {'true', 't', '1', 'yes', 'y'}:
        return True
    raise ValueError('{value} is not a valid boolean value')
print(torch.cuda.device_count())
DEVICE = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print(DEVICE)

0
cpu


In [2]:
case_name = "case30"
data_dir = "data/"
case_dir = os.path.join(data_dir, case_name)
bus_number = 30
sample_number = 20000

In [3]:
mat_input = os.path.join(data_dir, case_name, 'outputdata_sample.mat')
mat_ang = os.path.join(data_dir, case_name, 'voltage_ang.mat')
mat_mag = os.path.join(data_dir, case_name, 'voltage_mag.mat')

#voltage magnitude and angle
voltage_ang = np.transpose(scipy.io.loadmat(mat_ang)['voltage_ang'])[0:sample_number,:]
voltage_mag = np.transpose(scipy.io.loadmat(mat_mag)['voltage_mag'])[0:sample_number,:]
voltage = np.concatenate([voltage_ang,voltage_mag], axis=1) 
voltage_2d = np.dstack([voltage_ang,voltage_mag]) 
print(voltage_ang.shape)
print(voltage_mag.shape)
print(voltage.shape)
print(voltage_2d.shape)
print('the labels are angle and magnitude now')

#nodal power injection
PQ_injection = np.transpose(scipy.io.loadmat(mat_input)['power_injection'])[0:sample_number,:]
P_injection = PQ_injection.real
Q_injection = PQ_injection.imag

# PQ_injection = np.concatenate([P_injection, Q_injection], axis=1)
# PQ_injection = min_max_scaler.fit_transform(PQ_injection)
# P_injection = PQ_injection[:,0:30]
# Q_injection = PQ_injection[:,30:60]
PQ_injection_2d = np.dstack((P_injection,Q_injection)) 
print(PQ_injection_2d.shape)

(20000, 30)
(20000, 30)
(20000, 60)
(20000, 30, 2)
the labels are angle and magnitude now
(20000, 30, 2)


In [4]:
CASE_FNS = dict([(30, case30)])
ppc = CASE_FNS[30]()

In [5]:
# edge index, 2 * the number of edges
# the first list contains the index of the source nodes, 
# while the index of target nodes is specified in the second list.
edge_index = torch.tensor(ppc['branch'][:,0:2].transpose(), dtype=torch.long) - 1
edge_weight = torch.zeros(ppc['branch'].shape[0], dtype=torch.double)
print(edge_index.shape)
print(edge_weight.shape)

torch.Size([2, 41])
torch.Size([41])


In [6]:
def adjacency_from_net(node_number, scaling=1, threshold=0):
    N = node_number
    G = np.zeros((N, N))
    for i in range(ppc['branch'].shape[0]):
        from_node = int(ppc['branch'][i,0])-1
        to_node = int(ppc['branch'][i,1])-1
        impedance = np.array([ppc['branch'][i,2], ppc['branch'][i,3]])
        impedance_norm =  np.linalg.norm(impedance)
        value = np.math.exp(- (impedance_norm**2) * scaling)
        if value > threshold:
            edge_weight[i] = value
            G[from_node,to_node] = value
            G[to_node, from_node] = value
    return G, edge_weight

In [7]:
graph_adj, edge_weight = adjacency_from_net(node_number = bus_number, scaling=10, threshold=0.001)
# graph_adj = graph_adj.reshape([1,graph_adj.shape[0],graph_adj.shape[0]])
graph_adj.shape

(30, 30)

In [9]:
parser = argparse.ArgumentParser(description='GCN')
parser.add_argument('--epochs', type=int, default=200,
    help='number of neural network epochs')
parser.add_argument('--batchSize', type=int,  default=32,
    help='training batch size')
parser.add_argument('--lr', type=float, default=0.0001,
    help='neural network learning rate')
parser.add_argument('--inputChannel', type=int, default=2,
    help='input feature size for neural network')
parser.add_argument('--outputChannel', type=int, default=2,
    help='output feature size for neural network')
parser.add_argument('--hiddenSize', type=int, default=50,
    help='hidden layer size for neural network')
parser.add_argument('--hiddenSize_MLP', type=int, default=50,
    help='hidden layer size for neural network')
parser.add_argument('--hiddenLayer_MLP', type=int, default=1,
    help='hidden layer size for neural network')
parser.add_argument('--OutputSize_MLP', type=int, default=bus_number*2,
    help='hidden layer size for neural network')

args = parser.parse_args([])
args = vars(args) # change to dictionary
print(args)

{'epochs': 200, 'batchSize': 32, 'lr': 0.0001, 'inputChannel': 2, 'outputChannel': 2, 'hiddenSize': 50, 'hiddenSize_MLP': 50, 'hiddenLayer_MLP': 1, 'OutputSize_MLP': 60}


In [10]:
solver_step = args['lr']
nepochs = args['epochs']
batch_size = args['batchSize']

In [11]:
dataset = TensorDataset(torch.tensor(PQ_injection_2d), torch.tensor(voltage_2d))
valid_frac = 0.08
test_frac = 0.08
train_frac = 1 - valid_frac - test_frac
train_size = sample_number * train_frac

trainX = PQ_injection_2d[:int(sample_number * train_frac)]

validX = PQ_injection_2d[int(sample_number * train_frac):int(sample_number * (train_frac + valid_frac))]

testX = PQ_injection_2d[int(sample_number * (train_frac + valid_frac)):]

trainY = voltage[:int(sample_number * train_frac)]

validY = voltage[int(sample_number * train_frac):int(sample_number * (train_frac + valid_frac))]

testY = voltage[int(sample_number * (train_frac + valid_frac)):]

train_dataset = TensorDataset(torch.tensor(trainX), torch.tensor(trainY))
valid_dataset = TensorDataset(torch.tensor(validX), torch.tensor(validY))
test_dataset = TensorDataset(torch.tensor(testX), torch.tensor(testY))

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=len(valid_dataset))
test_loader = DataLoader(test_dataset, batch_size=len(test_dataset))

In [12]:
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
            
        self.model = Sequential('x, edge_index, edge_weight', [
            (GCNConv(args['inputChannel'], args['hiddenSize']), 'x, edge_index, edge_weight -> x'),
            ReLU(inplace=True),
            (GCNConv(args['hiddenSize'], args['hiddenSize']), 'x, edge_index, edge_weight -> x'),
            ReLU(inplace=True),
            (GCNConv(args['hiddenSize'], args['outputChannel']), 'x, edge_index, edge_weight -> x'),
        ])
        
        dimInputMLP = bus_number * args['outputChannel']
        
        self.MLP = nn.Sequential(
            nn.Linear(dimInputMLP, args['hiddenSize_MLP']),
            nn.ReLU(inplace=True),
            nn.Linear(args['hiddenSize_MLP'], args['OutputSize_MLP'])
        )   
        
    def forward(self, x, edge_index, edge_weight):
        
        h = self.model(x, edge_index, edge_weight)
        
        h = torch.flatten(torch.permute(h, (0, 2, 1)), start_dim=1)
        
        out = self.MLP(h)
        
        return out

In [15]:
solver_net = GCN().double()
solver_net.to(DEVICE)
solver_opt = optim.Adam(solver_net.parameters(), lr=solver_step)

stats = {}
batch_loss = []
epoch_loss = []

loss = nn.MSELoss()

In [ ]:
for i in range(nepochs):
    epoch_stats = {}

    # Get train loss
    solver_net.train()
    for Xtrain, Ytrain in train_loader:
        Xtrain = Xtrain.to(DEVICE)
        Ytrain = Ytrain.to(DEVICE)
        Yhat_train = solver_net(Xtrain, edge_index, edge_weight)
        
        solver_opt.zero_grad()
        
        # Compute loss
        train_loss = loss(Yhat_train, Ytrain)

        # Compute gradients
        train_loss.backward()

        # Optimize
        solver_opt.step()
        
        batch_loss.append(train_loss.detach().cpu().numpy())
        
    for Xvalid, Yvalid in valid_loader:
        Xvalid = Xvalid.to(DEVICE)
        Yvalid = Yvalid.to(DEVICE)
        Yhat_valid = solver_net(Xvalid, edge_index, edge_weight)
        valid_loss = loss(Yhat_valid, Yvalid)
    print(
        'Epoch {}: train loss {:.8f}, valid loss {:.8f},'.format(
            i, np.mean(batch_loss), valid_loss))
    
    epoch_loss.append(np.mean(batch_loss))
    batch_loss = []

Epoch 0: train loss 0.16173571, valid loss 0.00124389,


In [286]:
for Xtest, Ytest in test_loader:
    
    Xtest = Xtest.to(DEVICE)
    Ytest = Ytest.to(DEVICE)
    
    Yhat_test = solver_net(Xvalid, edge_index, edge_weight)

    test_loss = loss(Yhat_test, Ytest)